## Machine Learning

Perform the data analysis in a “batch” manner using machine learning to predict events such as days with high number of tickets (think of and implement at least one additional interesting learning problem). 

You will need to appropriately transform the augmented data. 

Ensure that each single worker will not have enough memory to store and process the entire dataset (e.g., 8GB per worker). Use at least three kinds of supervised machine learning algorithms:

1. One of the existing distributed algorithms from Dask-ML
2. A sophisticated third-party algorithm which “natively” supports distributed computing (such as XGBoost or LightGBM)
3. One of the common scikit-learn algorithms utilizing partial_fit.

For all three scenarios compare performance in terms of loss (error), scalability, time, and total memory consumption.

Note: scalability must be tested in the Arnes cluster by increasing the number of workers and
observing the total processing time.